In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [9]:
import argparse
import os

import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from tools.metrics import *

from eval import eval
from tools.utils import *
from tensorboardX import SummaryWriter

import re

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

In [3]:
sets = ['.1', '.2', '.5', '1', '2', '5']

with open('../configs/eval_carla_lss_evidential.yaml', 'r') as file:
    config = yaml.safe_load(file)

config['n_classes'] = 4

split = "mini"
dataroot = f"../../data/carla"

for s in sets:
    os.makedirs(f"../outputs/grid_gamma/hist/{s}")
    writer = SummaryWriter(logdir=f"../outputs/grid_gamma/hist/{s}")

    dl = sorted_alphanumeric(os.listdir(f"../outputs/grid_gamma/{s}"))
    for ch in dl:
        if ch.endswith(".pt"):
            path = os.path.join(f"../outputs/grid_gamma/{s}", ch)
            config['pretrained'] = path

            predictions, ground_truth, oods, aleatoric, epistemic = eval(config, False, 'val', split, dataroot)
            uncertainty_scores = aleatoric.squeeze(1)
            uncertainty_labels = torch.argmax(ground_truth, dim=1).cpu() != torch.argmax(predictions, dim=1).cpu()

            fpr, tpr, rec, pr, auroc, aupr, no_skill = roc_pr(uncertainty_scores, uncertainty_labels)
            e = ece(predictions, ground_truth)

            writer.add_scalar("hist/auroc", auroc, int(ch.split(".")[0]))
            writer.add_scalar("hist/aupr", aupr, int(ch.split(".")[0]))
            writer.add_scalar("hist/ece", e, int(ch.split(".")[0]))

NameError: name 'yaml' is not defined